In [17]:
'''
计算一次实验中的多路iFEMG信号的特征

更改4通道iFEMG采集板
1600采样率

没有计算主动与拮抗肌肉之间的差值'''

import pandas as pd

from utils_gui_model import form_sbj_info_df, form_feature_df_1ch
from utils_iFEMG_feature import df_save_csv

In [40]:
"""
初始化一些全局变量，只需运行一次
"""
feature_name_list = ['initial_pressure_ave', 'FMG', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']
sbj_info_name_list = ['subject_name', 'height', 'weight', 'age', 'gender', 'label']
sbj_info_col1_list = ['subject_info' for i in sbj_info_name_list]

ch1_str = "bicps_br"
ch2_str = "tricps_br_medial"
ch3_str = "tricps_br_lateral"

ch1_name_list = [ch1_str for i in feature_name_list]
ch2_name_list = [ch2_str for i in feature_name_list]
ch3_name_list = [ch3_str for i in feature_name_list]

# 存储文件名对应重量标签的dic
bicps_dic = {'bi-0': 0, 'bi-05': 0.5, 'bi-1': 1, 'bi-2': 2}
tricps_dic = {'tri-0': 0, 'tri-05': 0.5, 'tri-1': 1}

# feature_df = pd.DataFrame(columns=pd.MultiIndex.from_product([[ch1_str, ch2_str, ch3_str], feature_name_list]))
# sbj_info_df = pd.DataFrame(columns=pd.MultiIndex.from_product([['subject_info'], sbj_info_name_list]))
# all_feature_df = pd.concat([sbj_info_df, feature_df], axis = 1)
# print(all_feature_df)
# print("Init dataframe shape: ", all_feature_df.shape)

In [85]:

# 需要更改
# 更换受试需要更改
signal_channel_dic = {"bicps_br": 1,
                      "tricps_br_lateral": 2,
                      "tricps_br_medial": 4}
folder_path = r"E:\Data\paper2\积水潭患者数据集\1107s1\affected"
subject_name = "1107s1"
subject_args_dict = {"subject_name": subject_name,
                     "height": 150,
                    "weight": 60,
                    "age": 66,
                    "gender": 0}
fs = 1626   # 1626 or 1222
channel_nums = 4
# 更换动作需要更改
file_name_dic = bicps_dic

###########################################################################
# 以上都需要改
############################################################################

# 用于存储每个重量特征的临时df，最终合并成该动作下的最终df
temp_df_list = []
# key是文件名，value是对应的重量标签
# 循环弯举或伸展动作下的每个重量，计算3个肌肉的特征
for key, value in file_name_dic.items():
    # ch1_name_list: bicps_br
    temp_fea_df1 = form_feature_df_1ch(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["bicps_br"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = fs, 
                                    ch_nums=channel_nums)
    if temp_fea_df1 is not None:
        temp_fea_df1.columns = [ch1_name_list, feature_name_list]
    else:
        print(f"跳过当前循环")
        continue    # 如果为空则跳过当前循环，不产生temp_df
    # ch2_name_list: tricps_med
    temp_fea_df2 = form_feature_df_1ch(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["tricps_br_medial"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = fs,
                                    ch_nums=channel_nums)
    if temp_fea_df2 is not None:
        temp_fea_df2.columns = [ch2_name_list, feature_name_list]
        pass
    # ch3_name_list: tricps_lat
    temp_fea_df3 = form_feature_df_1ch(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["tricps_br_lateral"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = fs,
                                    ch_nums=channel_nums)
    if temp_fea_df3 is not None:
        temp_fea_df3.columns = [ch3_name_list, feature_name_list]
        pass

    # 受试者个人信息和重量label
    temp_sbjinfo_df = form_sbj_info_df(df_len = len(temp_fea_df1), **subject_args_dict, label = value)
    temp_sbjinfo_df.columns = [sbj_info_col1_list, sbj_info_name_list]
    
    # 合并df
    temp_df = pd.concat([temp_sbjinfo_df, temp_fea_df1, temp_fea_df2, temp_fea_df3], axis=1)
    print(f"特征数量：{temp_df.shape}")
    temp_df_list.append(temp_df)
    # all_feature_df = pd.concat([all_feature_df, temp_df], axis = 0, ignore_index=True)
    # print(all_feature_df.shape)
    pass

all_feature_df = pd.concat(temp_df_list, axis = 0, ignore_index=True)
print(all_feature_df.shape)
all_feature_df.head


特征数量：(8, 33)
特征数量：(8, 33)
特征数量：(8, 33)
特征数量：(8, 33)
(32, 33)


<bound method NDFrame.head of    subject_info                                            bicps_br  \
   subject_name height weight age gender label initial_pressure_ave   
0        1107s1    150     60  66      0   0.0           472.844921   
1        1107s1    150     60  66      0   0.0           472.844921   
2        1107s1    150     60  66      0   0.0           472.844921   
3        1107s1    150     60  66      0   0.0           472.844921   
4        1107s1    150     60  66      0   0.0           472.844921   
5        1107s1    150     60  66      0   0.0           472.844921   
6        1107s1    150     60  66      0   0.0           472.844921   
7        1107s1    150     60  66      0   0.0           472.844921   
8        1107s1    150     60  66      0   0.5           628.809597   
9        1107s1    150     60  66      0   0.5           628.809597   
10       1107s1    150     60  66      0   0.5           628.809597   
11       1107s1    150     60  66      0   0.5 

In [86]:
"""
写入文件，增加覆盖检查功能
运行前更改：
    1. 肌肉名称
"""
filename = folder_path + '\\' + subject_name + "_curl_abs.csv"

df_save_csv(all_feature_df, filename)

File E:\Data\paper2\积水潭患者数据集\1107s1\affected\1107s1_curl_abs.csv saved!


In [69]:
"""
写入文件，增加覆盖检查功能
运行前更改：
    1. 肌肉名称
"""
filename = folder_path + '\\' + subject_name + "_extension_abs.csv"

df_save_csv(all_feature_df, filename)

File E:\Data\paper2\积水潭患者数据集\1010s1\affected\1010s1_extension_abs.csv saved!
